In [26]:
import arcpy
import pandas as pd
from difflib import get_close_matches
import pickle

lyr_jurisdictions = r'C:\Users\daniel.fourquet\Documents\ArcGIS\Basemap Layers\basemap.gdb\all_jurisdictions'
contacts_list = r'C:\Users\daniel.fourquet\Documents\Tasks\VTrans Update\Mid-Term Needs\Urban Development Areas (UDAs) Needs\Survey Email Template\UDA_Contacts (2).xlsx'

In [10]:
# Get coordinates for each jurisdiction in the state

arcpy.env.overwriteOutput = True
lyr_jurisdictions_prj = r'C:\Users\daniel.fourquet\Documents\ArcGIS\Basemap Layers\basemap.gdb\all_jurisdictions_prj'
arcpy.Project_management(lyr_jurisdictions, lyr_jurisdictions_prj, arcpy.SpatialReference(4326))
jurisdiction_coords = {row[0]: row[1] for row in arcpy.da.SearchCursor(lyr_jurisdictions_prj, ['NAMELSAD', 'SHAPE@XY'])}

In [27]:
# Lookup coordinates for each UDA jurisdiction
df = pd.read_excel(contacts_list)
uda_jurisdictions = df['Jurisdiction'].unique()
uda_jurisdictions = list(uda_jurisdictions)

uda_jurisdiction_match = {}
for juris in uda_jurisdictions:
    match = get_close_matches(juris, jurisdiction_coords, n=1, cutoff=0.25)[0]
    uda_jurisdiction_match[juris] = match

# Known error fix
uda_jurisdiction_match['Town of Luray'] = 'Page County'

# Build URL
uda_jurisdiction_urls = {}
for juris in uda_jurisdictions:
    match = uda_jurisdiction_match[juris]
    lng, lat = jurisdiction_coords[match]
    url = f'https://vtrans.org/interactvtrans/map-explorer?layer=Urban%20Development%20Areas&field=Jurisdiction&center={lng}%2C{lat}&zoom=11'
    uda_jurisdiction_urls[juris] = url

# Save coords
with open('urls.pkl', 'wb') as file:
    pickle.dump(uda_jurisdiction_urls, file)

In [29]:
uda_jurisdiction_urls['Chesterfield County']

'https://vtrans.org/interactvtrans/map-explorer?layer=Urban%20Development%20Areas&field=Jurisdiction&center=-77.58615333266718%2C37.37859609584751&zoom=11'